In [1]:
import dask.dataframe as dd
import dask.bag as db
from dask.distributed import Client
import os
import json
import logging
import subprocess

In [2]:
DATA_DOWNLOAD_DIRECTORY = "/home/r0835817/2023-WoutRombouts-NoCsBack/ml4see/download"
DATA_SUMMARY_PATH = "/home/r0835817/2023-WoutRombouts/ml4see/data_retrieval_download.json"

In [3]:
logging.basicConfig(filename='data_retrieval_verify.log', filemode="w", level=logging.INFO, format='%(asctime)s.%(msecs)03d %(levelname)s %(module)s - %(funcName)s: %(message)s')

In [4]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 32,Total memory: 125.69 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38329,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 32
Started: Just now,Total memory: 125.69 GiB
Comm: tcp://127.0.0.1:40319,Total threads: 4
Dashboard: http://127.0.0.1:44495/status,Memory: 15.71 GiB
Nanny: tcp://127.0.0.1:33063,


In [5]:
if not os.path.exists(DATA_DOWNLOAD_DIRECTORY):
    logging.warning("The data download directory does not exist at {}.".format(DATA_DOWNLOAD_DIRECTORY))
    
if not os.path.exists(DATA_SUMMARY_PATH):
    logging.warning("The data summary file does not exist at {}.".format(DATA_SUMMARY_PATH))

In [6]:
runs = []
with open(DATA_SUMMARY_PATH, 'r', encoding="utf-8") as file:
    runs = json.load(file)

In [7]:
def validate_file(run):
    file_path = os.path.join(DATA_DOWNLOAD_DIRECTORY, run["name"] + ".tar")
    if not os.path.exists(file_path):
        logging.warning("Skip validating {} since it was not found on disk!".format(run["name"]))
        return
    
    if "md5sum" not in run:
        logging.warning("Skip validating {} since it has no md5 checksum available!".format(run["name"]))
        return
        
    result = subprocess.run(['md5sum', file_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, check=True)
    md5_returned = result.stdout.split()[0]
        
    if not md5_returned == run["md5sum"]:
        logging.error("Validation of {} failed!".format(run["name"]))
        logging.error("Got {} as checksum but expected {}.".format(md5_returned, run["md5sum"]))
        return

    logging.info("Validation of {} succeeded.".format(md5_returned, run["md5sum"]))
    return

bag = db.from_sequence(runs)
tasks = bag.map(validate_file)

In [8]:
#tasks.visualize()

In [ ]:
results = tasks.compute()

ERROR:root:Validation of run_044 failed!
ERROR:root:Got b3f30cd33c059f3bb7b26ba7475a8f02 as checksum but expected bbc36c4c4d1612ee994160f0024cc91d.
ERROR:root:Validation of run_042 failed!
ERROR:root:Got 9800468e4d48c92dfaf19debb3670c7a as checksum but expected 605556b3ecbbe18919abf6e869a5ed15.


In [ ]:
client.close()